In [40]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time 
from selenium.webdriver.support.ui import Select
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [41]:
all_data = []

In [42]:

# Set up Chrome options for headless mode and different user agent
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Create a webdriver instance with the specified options
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the desired website
url = 'https://www.evetech.co.za/components.aspx'
driver.get(url)

In [43]:
def scrape_site(driver):
    time.sleep(2)
    # Get the page source (HTML content)
    page_source = driver.page_source

    # Use BeautifulSoup to parse the HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    #Product category
    product_category = soup.find('div', class_='detail')
    product_category = product_category.find('h1').text
    # print(product_category)

    # Find the number of product
    products = soup.findAll('div', class_='ComponentCard_Products__Card__SJT5q ComponentCard_HoverGrow__Q2lEZ shadow overflow-hidden h-100 gap-2 position-relative card')
    print("Number of products:", len(products))

    for i in products:
        product_names = i.find('h3', class_="fs-6 fw-2 lh-1 m-0 overflow-hidden h-100").text
        product_price = int(i.find('div', class_="ComponentCard_Products__Price__SG2mn fw-3 fs-3 flex-shrink-0").text.replace("R ", "").strip())
        product_availiablity = True if i.find('span', class_="fw-1 fs-6 text-wrap").text == "In Stock with Evetech" else False
        all_data.append([product_names, product_price, product_availiablity, product_category])
        # print(product_names, ": ", product_price, ": ", product_availiablity,": ", product_category)


In [44]:
# Get the page source (HTML content)
page_source = driver.page_source

# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(page_source, 'html.parser')

#Find all categories button 
categories = soup.findAll('div', class_='Components_Child__mYntX Components_HoverGrow__br6Zs position-relative')
print("Number of categories:", len(categories))
page_positioin = 200
for i, category in enumerate(categories):
    # Find all buttons within the category div
    buttons = category.find_all('button', class_='rounded-pill bg-gradient lh-1 border border-primary btn btn-light btn-sm')

    #Check if there is a button with view all
    button_view_all = category.find('button', class_='rounded-pill bg-gradient lh-1 btn btn-primary btn-sm')

    if button_view_all is not None:
        get_number_of_buttons = len(buttons)
        # You can construct XPath using the button class if needed
        button_xpath = f'//*[@id="root"]/div/div[2]/div/section[2]/div/div/div[{i+1}]/div[1]/div[1]/button[{get_number_of_buttons+1}]'

        # Click the button
        driver.find_element(By.XPATH, button_xpath).click()
        print("Scraping:", driver.current_url)
        scrape_site(driver)
        time.sleep(3)

        #Go back to the previous page
        driver.execute_script("window.history.go(-1)")
        time.sleep(5)
        driver.execute_script(f"window.scrollTo(0, {str(page_positioin)});")
        time.sleep(5)
    else:
        for k, button in enumerate(buttons):
            # Print the text content of the button
            print("Button Text:", button.text)

            # You can construct XPath using the button class if needed
            button_xpath = f'//*[@id="root"]/div/div[2]/div/section[2]/div/div/div[{i+1}]/div[1]/div[1]/button[{k+1}]'

            # Click the button
            driver.find_element(By.XPATH, button_xpath).click()
            print("Scraping:", driver.current_url)
            scrape_site(driver)
            time.sleep(3)

            #Go back to the previous page
            driver.execute_script("window.history.go(-1)")
            time.sleep(5)
            driver.execute_script(f"window.scrollTo(0, {str(page_positioin)});")
            time.sleep(5)
    if (i+1) % 3 == 0:
        page_positioin += 300
        
        
    i += 1



Number of categories: 35
Button Text: Case Fans
Scraping: https://www.evetech.co.za/PC-Components/case-fans-97.aspx
Number of products: 72
Scraping: https://www.evetech.co.za/components/computer-case-accessories-75.aspx
Number of products: 9
Button Text: Fractal Design Cases
Scraping: https://www.evetech.co.za/PC-Components/fractal-design-cases-346.aspx
Number of products: 26
Button Text: ASUS Gaming Cases
Scraping: https://www.evetech.co.za/PC-Components/asus-gaming-cases-336.aspx
Number of products: 1
Button Text: Gamdias Gaming Cases
Scraping: https://www.evetech.co.za/PC-Components/gamdias-gaming-cases-293.aspx
Number of products: 29
Button Text: Evetech Gaming Cases
Scraping: https://www.evetech.co.za/PC-Components/evetech-gaming-cases-287.aspx
Number of products: 9
Button Text: Antec Gaming Cases
Scraping: https://www.evetech.co.za/PC-Components/antec-gaming-cases-220.aspx
Number of products: 25
Button Text: Corsair Gaming Cases
Scraping: https://www.evetech.co.za/PC-Components/c

KeyboardInterrupt: 

In [ ]:
# Close the browser
driver.quit()

# Write the data to csv file
with open('evetech.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Product Name', 'Product Price', 'Product Availability', 'Product Category'])
    writer.writerows(all_data)

STOPPED Here

In [ ]:


# # Set up Chrome options for headless mode and different user agent
# chrome_options = Options()
# chrome_options.add_argument('--headless')  # Enable headless mode
# chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# # Create a webdriver instance with the specified options
# driver = webdriver.Chrome(options=chrome_options)

# # Navigate to the desired website
# url = 'https://www.evetech.co.za/components/gaming-chairs-south-africa-212.aspx'
# driver.get(url)

# # Get the page source (HTML content)
# page_source = driver.page_source

# # Use BeautifulSoup to parse the HTML
# soup = BeautifulSoup(page_source, 'html.parser')

# #Product category
# product_category = soup.find('div', class_='detail')
# product_category = product_category.find('h1').text
# print(product_category)

# # Find the number of product
# products = soup.findAll('div', class_='ComponentCard_Products__Card__SJT5q ComponentCard_HoverGrow__Q2lEZ shadow overflow-hidden h-100 gap-2 position-relative card')
# print("Number of products:", len(products))

# for i in products:
#     product_names = i.find('h3', class_="fs-6 fw-2 lh-1 m-0 overflow-hidden h-100").text
#     product_price = i.find('div', class_="ComponentCard_Products__Price__SG2mn fw-3 fs-3 flex-shrink-0").text
#     product_availiablity = True if i.find('span', class_="fw-1 fs-6 text-wrap").text == "In Stock with Evetech" else False
#     print(product_names, ": ", product_price, ": ", product_availiablity,": ", product_category)

# # Close the browser window
# driver.quit()
